**复习：**在前面我们已经学习了Pandas基础，第二章我们开始进入数据分析的业务部分，在第二章第一节的内容中，我们学习了**数据的清洗**，这一部分十分重要，只有数据变得相对干净，我们之后对数据的分析才可以更有力。而这一节，我们要做的是数据重构，数据重构依旧属于数据理解（准备）的范围。

#### 开始之前，导入numpy、pandas包和数据

In [2]:
# 导入基本库
import pandas as pd
import numpy as np

In [3]:
# 载入上一个任务人保存的文件中:result.csv，并查看这个文件
df = pd.read_csv('result.csv')
df.head(3)

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


# 2 第二章：数据重构


## 第一部分：数据聚合与运算

### 2.6 数据运用

#### 2.6.1 任务一：通过教材《Python for Data Analysis》P303、Google or anything来学习了解GroupBy机制

#### 2.4.2：任务二：计算泰坦尼克号男性与女性的平均票价

In [14]:
# 写入代码
df['Fare'].groupby(df['Sex']).mean() #groupby如果前面不是整个df，后面就得加df

Sex
female    44.479818
male      25.523893
Name: Fare, dtype: float64

在了解GroupBy机制之后，运用这个机制完成一系列的操作，来达到我们的目的。

下面通过几个任务来熟悉GroupBy机制。

#### 2.4.3：任务三：统计泰坦尼克号中男女的存活人数

In [25]:
a = df['PassengerId'].groupby([df['Survived'], df['Sex']]).count() ##前面不是整个df所以后面得加df不能简写
#或者 
b = df.groupby(['Survived','Sex']).size().unstack()  ##这个最好，注意size包括非空值，而count不包括非空值

c = df['Survived'].groupby(df['Sex']).sum()  ##因为是dummy variable所以可以直接sum
print(
    a,
    b,
    c
)

Survived  Sex   
0         female     81
          male      468
1         female    233
          male      109
Name: PassengerId, dtype: int64 Sex       female  male
Survived              
0             81   468
1            233   109 Sex
female    233
male      109
Name: Survived, dtype: int64


#### 2.4.4：任务四：计算客舱不同等级的存活人数

In [28]:
d = df['Survived'].groupby(df['Pclass']).sum()
e = df.groupby(['Survived','Pclass']).size().unstack()
print(d,e)

Pclass
1    136
2     87
3    119
Name: Survived, dtype: int64 Pclass      1   2    3
Survived              
0          80  97  372
1         136  87  119


【**提示：**】表中的存活那一栏，可以发现如果还活着记为1，死亡记为0

【**思考**】从数据分析的角度，上面的统计结果可以得出那些结论

We can find that higher Pclass lead to lower death rate and female have a much higher survival rate than man. 


【思考】从任务二到任务三中，这些运算可以通过agg()函数来同时计算。并且可以使用rename函数修改列名。你可以按照提示写出这个过程吗？

In [33]:
#法一
grouped = df.groupby('Sex')
mission2 = grouped['Fare'].agg('mean')
mission3 = grouped['Survived'].agg('sum')
print(
    mission2,
    mission3
)
grouped

Sex
female    44.479818
male      25.523893
Name: Fare, dtype: float64 Sex
female    233
male      109
Name: Survived, dtype: int64


In [36]:
#法二，全写进一个
df.groupby('Sex').agg({'Fare': 'mean', 'Survived': 'sum','Pclass': 'count'}).rename(
    columns={
        'Fare':'mean_fare',
        'Survived':'survived_sex',
        'Pclass':'Pclass_survival'
})  #注意columns有s

,mean_fare,survived_sex,Pclass_survival
Sex,,,
female,44.479818,233,314
male,25.523893,109,577


#### 2.4.5：任务五：统计在不同等级的票中的不同年龄的船票花费的平均值

In [40]:
## 法一
grouped_5 = df.groupby(['Pclass','Age'])
grouped_5['Fare'].mean().unstack()

## 法二
df.groupby(['Pclass','Age'])['Fare'].mean().head() #注意这里【fare】放后面是特殊写法

Age,0.42,0.67,0.75,0.83,0.92,1.00,2.00,3.00,4.00,5.00,...,62.00,63.00,64.00,65.00,66.00,70.00,70.50,71.00,74.00,80.00
Pclass,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,151.55,NaN,151.550000,NaN,81.858300,NaN,...,44.366667,77.9583,144.5,44.2646,NaN,71.0,NaN,42.0792,NaN,30.0
2,NaN,14.5,NaN,23.875,NaN,38.0021,26.000000,28.7764,31.000000,27.750000,...,10.500000,NaN,NaN,NaN,10.5,10.5,NaN,NaN,NaN,NaN
3,8.5167,NaN,19.2583,NaN,NaN,26.8075,24.544643,22.7875,21.653571,21.040267,...,NaN,9.5875,NaN,7.7500,NaN,NaN,7.75,NaN,7.775,NaN


#### 2.4.6：任务六：将任务二和任务三的数据合并，并保存到sex_fare_survived.csv

In [42]:
# 写入代码
m2 = df['Fare'].groupby(df['Sex']).mean()
m3 = df['Survived'].groupby(df['Sex']).sum()
print(m2,'\n',m3,sep='')
pd.merge(m2,m3,
        left_index=True,right_index=True) #或者这里可以写 on=sex，因为index已经有名字了

Sex
female    44.479818
male      25.523893
Name: Fare, dtype: float64
Sex
female    233
male      109
Name: Survived, dtype: int64


,Fare,Survived
Sex,,
female,44.479818,233
male,25.523893,109


#### 2.4.7：任务七：得出不同年龄的总的存活人数，然后找出存活人数最多的年龄段，最后计算存活人数最高的存活率（存活人数/总人数）


In [46]:
grouped_age = df.groupby('Age')
survival_age = grouped_age['Survived'].sum()


使用 idxmax() 找出存活人数最多的年龄段。
使用 max() 获取存活人数最多的数量。

In [48]:
print(survival_age.idxmax())
survival_age.max()


24.0


15

In [52]:
total_survival = df['Survived'].sum()
survival_rate = survival_age.max() / total_survival
survival_rate


0.043859649122807015